In [1]:
#standard libraries
import pandas as pd
import numpy as np
import re
import pickle

#sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

#nltk
import nltk
from nltk.corpus import stopwords
from collections import Counter
from langdetect import detect

#preprocessing and feature extraction
import helper

#tensorflow
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.utils import FeatureSpace

from tensorflow import feature_column as fc
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import random
random.seed(42)

In [2]:
df = pd.read_csv('data/online_ship_listing_data.csv')

In [3]:
#df = helper.preprocess_df(df)

In [4]:
#df_test = df.copy()

In [5]:
#df_test = helper.create_features(df_test)

In [6]:
#splitting the dataframe
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

1184 train examples
296 validation examples
370 test examples


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1184 entries, 1750 to 667
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     1184 non-null   int64  
 1   name           1184 non-null   object 
 2   price          1184 non-null   float64
 3   location       1184 non-null   object 
 4   offerer        1184 non-null   object 
 5   id             1184 non-null   int64  
 6   description    1184 non-null   object 
 7   year           1184 non-null   int64  
 8   manufacturer   1184 non-null   object 
 9   model          1179 non-null   object 
 10  category       1184 non-null   object 
 11  length         1184 non-null   float64
 12  fuel_type      1097 non-null   object 
 13  hull_material  1184 non-null   object 
 14  hull_shape     258 non-null    object 
 15  country        1184 non-null   object 
dtypes: float64(2), int64(3), object(11)
memory usage: 157.2+ KB


In [8]:
def df_to_dataset(df, shuffle=True, batch_size=32):
    df = df.copy()
    
    df = helper.preprocess_df(df)
    df, one_hot_cols = helper.create_features(df)
    labels = df.pop('price')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds, one_hot_cols

In [9]:
#convert dataframe to dataset
batch_size = 32
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

C:\Online Course\Projects\shipping-listing-price-prediction\helper.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['word_count'], axis=1, inplace=True)
C:\Online Course\Projects\shipping-listing-price-prediction\helper.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description_language'] = df['description'].apply(lambda x:detect(x))
C:\Online Course\Projects\shipping-listing-price-prediction\helper.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of item_name:', feature_batch['ITEM_NAME'])
    print('A batch of quantity:', feature_batch['PRICE'])
    print('A batch of holiday:', feature_batch['HOLIDAY'])
    print('A batch of is_schoolbreak:', feature_batch['IS_SCHOOLBREAK'])
    print('A batch of month:', feature_batch['MONTH'])
    print('A batch of day:', feature_batch['DAY'])
    print('A batch of targets:', label_batch)